In [ ]:
!nvidia-smi

In [ ]:
!pip install --upgrade jupyter

In [ ]:
!pip install -q transformers

In [ ]:
!pip install ipywidgets

In [ ]:
!pip install ipywidgets widgetsnbextension pandas-profiling

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
!pip install -U scikit-learn

In [ ]:
!pip install tensorboard

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from transformers import AdamW
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
class JpSentiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
df = pd.read_csv("train35404hard_sents.tsv", sep='\t', header=None,
                 names=["id", "text1", "text2","last_text", "label", "usage"])
df["text"] = df["text1"] + "[SEP]" + df["text2"] + "[SEP]" + df["last_text"]

df_train = df[(df["usage"] == "train")]
temp1 = df_train["text"]
train_docs = temp1[:].tolist()
temp2 = df_train["label"]
train_labels = temp2[:].tolist()
print(len(train_docs))

In [ ]:
df_dev = pd.read_csv("1000_dev2.tsv", sep='\t', header=None,
                 names=["id", "text1", "text2","last_text", "label", "usage"])
df_dev["text"] = df_dev["text1"] + "[SEP]" + df_dev["text2"] + "[SEP]" + df_dev["last_text"]

df_dev1 = df_dev[(df_dev["usage"] == "test")]
temp3 = df_dev1["text"]
dev_docs = temp3[:].tolist()
temp4 = df_dev1["label"]
dev_labels = temp4[:].tolist()
print(len(dev_docs))

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
model_name = "cl-tohoku/bert-large-japanese"
# model_name = "cl-tohoku/bert-base-japanese-v2"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model = model.to(device)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
train_encodings = tokenizer(train_docs, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
dev_encodings = tokenizer(dev_docs, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
train_dataset = JpSentiDataset(train_encodings, train_labels)
dev_dataset = JpSentiDataset(dev_encodings, dev_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_2/5e-7_16_epoch10",  # output directory
    num_train_epochs=10,  # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,  # batch size for evaluation
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch.
    dataloader_pin_memory=False,
    save_strategy="epoch",
    logging_dir="./logs_2/5e-7_16_epoch10",
    logging_strategy="epoch",
    tf32=True,
    learning_rate=5e-7
)

In [ ]:
trainer = Trainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=dev_dataset,  # evaluation dataset
    compute_metrics=compute_metrics  # The function that will be used to compute metrics at evaluation
)

In [ ]:
trainer.train()

trainer.train()

In [ ]:
save_dir = "./step2_5e-7_epoch10_16"
tokenizer.save_pretrained(save_dir)
model.save_pretrained(save_dir)